# Predictive Analtics Exam 2

### 1. True or False

a) Random forest is not an ensemble learning technique. FALSE

b) Support vector machine is an ensemble learning technique. TRUE

c) If the classifier produce a 80% recall on test dataset (using default values for hyperparameters), there is no need to tune the classifier hyper-parameters. FALSE

d) Random search and grid search always produce the same results. FALSE

e) Increasing the value of max depth in tree-based models prevents overfitting. FALSE

f) Scaling the input variables in tree-based models can speed up the hyper-parameter tuning process. FALSE

g) Random search is preferred when the training dataset is big and the number of hyperparameter combinations is large. TRUE

h) XGBoost always outperforms random forest. FALSE

i) XGBoost is faster than regular gradient boosting algorithms because it trains the the base-learners in parallel. 

j) In order to avoid overfitting, it is recommended to tune the learning rate hyperparameter in random forest models. FALSE (not applicable for RF)

### 2. How does stacking work? Be specific.

Stacking is an ensemble machine learning algorithm that involves combining the predictions from multiple machine learning models on the same dataset in order to create a meta-learner. A meta-learner takes the individual model predictions as inputs and the actual responses as outputs to train a higher-level
learner. Essentially, stacking uses individual model predictions to create a meta-learner that can provide increased prediction performance, no matter how good the individual models performed. 

### 3.  If you have trained three different models on the exact same training dataset and they all achieve a 93% F1-score on the exact same dataset, is there any chance that you can combine these three models to get better results? If so, how? If not, why? Be specific.

Yes, you could still combine these three models to potentially get better results. By extracting the test data-frame likelihoods from each of the considered model types, you can create an ensemble learning model which takes these likelihoods as input variables to predict the class. Even if all thre models have already achieved an F1-Score of 93%, it is still possible for an ensemble model to improve prediction power because it can find new patterns in the estimated likelihoods and produce its own. 

### 4. If your AdaBoost model overfits the data (probably because the number of boosted trees is large), what hyper-parameter you need to tune? Be specific.

If the AdaBoost model is overfitting the data with a large number of boosted trees, then the learning rate hyper-parameter needs to be tuned (lowered). In lowering the learning rate, less importance will be placed on the previous batch of boosted trees, and the model will be able to better generalize on new data. In general, decreasing learning rate prevents overfitting. 

### 5. Which of the following algorithm is NOT an ensemble learning algorithm?

(a) support vector machine is not an ensemble learning algorithm

### 6. How does XGBoost with random forest as base learner work? Be specific.

Random Forest is not an option for the 'booster' hyper-parameter in XGBoost, but it can be implemented by setting the 'num_parallel_trees' hyper-parameter to an integer greater than 1. Essentially what this does is it makes the model no longer consider only one tree for each boosting round (default 'num_parallel_trees' value = 1), but rather a number of parallel trees (the same principle as a Random Forest model). With each boosting round consisting of say, 500, num_parallel_trees, it has the same effect as using a Random Forest with 500 trees as the XGBoost base learner. 

### 7. How do we select the best hyper-parameter combination for a given model?

(b) Performance on the validation dataset

### 8. The below chart shows the performance of a random forest models (with different number of trees). Using the above chart, answer parts (a)-(b). 

### (a) If a data scientist wants to tune the number of trees based on the F1-score, how many trees would he select? Be specific. 
To tune the number of trees based on the F1-score, the data scientist would select 400 trees because there is little to no prediction improvement in the training and testing sets after increasing the number of trees past 400. Anything more would result in an increased computation time with no model improvement.

### (b) What is the reason there is big difference in the F1-score performance between the Train and Test datasets? Be specific.
The main reason why there would be a huge difference in the F1-score performance between the train and test datasets is that the model is overfitting the data. Typically with hyper-parameter tuning, we create a model that fits the training data very well, but does not do a great job of generalizing on the test set. This results in a huge performance loss on the testing set. Another reason could be that the test data set is not representative of the training set which would also decrease test performance, but this is much less likely to be the root error compared to training an over-fit model.

### 9.  What is the difference between homogeneous and heterogeneous ensembles? Be specific.

The difference between homogeneous and heterogeneous ensembles is the variety of modeling algorithms that are used to ensemble the predictions. In homegeneous ensembles, the same algorithm is used for all base learners on different distributions of the data set (i.e. using different subsets of the data to build a new learner). An example of this would be combining the predictions of three different Random Forest models built on three distinct subsets of the data. On the other hand, heterogeneous ensembles are created using multiple different algorithms as base learners. An example of this is combining the predictions of a Random Forest model, Support Vector Machine model, and XGBoost model on the same data set. Homogeneous uses only one algorithm, heterogeneous considers multiple. 

### 10 Random forest is an example of
(d) (a) Homogeneous ensemble and (c) Bagging

## 11. A junior data scientist is working on building a classification framework that can help a small local credit union to identify fraudulent transactions. The data scientist collects a very small dataset with 100 observations, from which 4 observations are labelled as fraudulent, to start building and tuning the model. As part of the tuning process, the data scientist considers using the GridSearch function from scikit-learn with cv = 5. Explain why this approach is a good idea or not. Be specific.

This sounds like a very poor idea. The GridSearchCV framework uses a stratified k-fold approach, which means that by setting cv = 5, the framework splits the complete 100 observation set into five data sets that are meant to contained a stratified sample of the class of interest (fraud transaction in this case). However, since there are not even five positive fraud cases, the GridSearchCV function would be (a) unable to split the data for the purpose of cross validation, or (b) able to split the data but one of the resulting data sets would have no positive fraud cases. Whichever one of these options is the case, it is a bad idea all around to use the GridSearch function in this scenario. 

### 12. The typical hyper-parameters in a XGBoost model that data science practitioners tune are: n estimators, learning rate, max depth, min child weight, subsample, and colsample bytree.

### (a) What is/are the hyper-parameters that may cause overfitting if we increase their values while holding the values of the hyper-parameters constant? Be specific.
Of the considered hyper-parameters, if we increase the values for learning rate, max depth, subsample, and colsample bytree while holding all other values constant, we may see overfitting of the model. Increasing these specific hyper-parameters increases the complexity of the model and increases the model's reliance on the training data observations. This is a recipe for overfitting. 


### (b) What is/are the hyper-parameters that prevents overfitting if we increase their values while holding the values of the hyper-parameters constant? Be specific.
Of the considered hyper-parameters, if we increase the value for min child weight while holding all other values constant, we may see prevention against overfitting of the model. By increasing this hyper-parameter, the number of splits in the data is limited and therefore prevents against overfitting of the data. 

### 13. In tree-based models, how does a data scientist optimize the number of trees? Be specific.

For tree-based models, a data scientist can optimize the number of trees by tuning it as a hyper-parameter. For tree-based models such as AdaBoost or Random Forest, n_estimators represents the number of trees to be considered and this could be tuned within the GridSearch framework. 

### 14. A Machine Learning Specialist is assigned to a team that is responsible for tuning an XGBoost model to ensure that it performs correctly on test data. However, when dealing with unknown data, this does not operate as planned. The following table summarizes the existing hyper-parameters:

XGBoost_params = {’n_estimators’: 2000, ’max_depth’: 30, ’min_child_weight’: 3, ’subsample’: 0.9, ’objective’: ’reg:squarederror’}

### (a) What type of task the machine learning specialist is working on? Classification or regression? Be specific.
The machine learning specialist is working on a regression task. "reg:squarederror" is the default setting for the objective hyper-parametering in an XGBoost regression task. 

### (b) Given the existing XGBoost hyper-parameters, what may be the reason that the XGBoost model is not performing as expected? Be specific.
Given the existing hyper-parameters for the XGBoost model, it may not be performing as expected because the n_estimators and max_depth values are promoting a model that is overfit. Using 2000 trees with a max depth of 30 is just a recipe for overfitting the training data. Although increasing the number of trees may improve the result, 2000 may be too many to consider. Similarly, decreasing the max depth will prevent the likelihood that the model is overfit. Overall, the "Specialist" should consider changes to n_estimators and max_depth to see some increased model performance.  

### 15. Considering the train.csv and test.csv data files containing information on default payments, demographic factors, credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005. The goal is to predict default payment next month on the test.csv data file

In [ ]:
pip install XGBoost

In [1]:
## Importing necessary libraries

import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import numpy as np
import matplotlib.pyplot as plt
import precision_recall_cutoff_exam2 as prc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

In [2]:
## a) Using the pandas library to read the train.csv and test.csv data files and create two data-frames called train and test

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

file_key = 'train(1).csv'
file_key2 = 'test(1).csv'

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')

train = pd.read_csv(file_content_stream)
test = pd.read_csv(file_content_stream2)

train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,400000,1,1,2,32,0,0,0,0,0,0,55773,55917,51389,48272,49478,51242,3028,3023,3000,3000,3000,38662,0
1,120000,2,2,2,30,-1,-1,-1,-1,-1,-1,140,3230,3011,1964,1883,1538,3230,3011,1964,1883,1538,1911,0
2,270000,2,2,2,32,0,0,0,0,0,0,59710,49986,104390,94856,86461,83650,1808,69563,2891,2689,3012,2771,0
3,280000,2,2,1,27,0,0,0,0,0,0,280913,283222,273160,257689,193231,191143,11052,9563,15017,5374,5420,6021,0
4,30000,2,1,2,27,0,0,-1,0,0,-2,1512,2458,664,1814,0,0,1000,664,1500,0,0,0,0


In [3]:
## Engineering features from Exam 1

## Train set:

## Most common repayment status
train['Most_Common'] = np.nan
for i in range(0, train.shape[0]):
    train.at[i, 'Most_Common'] = train[['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']].loc[i].mode()[0]

## From plot tree:
train['Tree2'] = np.where((train['PAY_0'] <= 1.5) & (train['PAY_2'] <= 1.5) & (train['PAY_AMT3'] > 395.0), 1, 0)
train['Tree6'] = np.where((train['PAY_0'] > 1.5) & (train['PAY_6'] <= 1.0) & (train['BILL_AMT1'] > 649.5), 1, 0)
train['Tree7'] = np.where((train['PAY_0'] > 1.5) & (train['PAY_6'] > 1.0) & (train['PAY_AMT3'] <= 14177.0), 1, 0)



## Test set:

## Most common repayment status
test['Most_Common'] = np.nan
for i in range(0, test.shape[0]):
    test.at[i, 'Most_Common'] = test[['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']].loc[i].mode()[0]

## From plot tree:
test['Tree2'] = np.where((test['PAY_0'] <= 1.5) & (test['PAY_2'] <= 1.5) & (test['PAY_AMT3'] > 395.0), 1, 0)
test['Tree6'] = np.where((test['PAY_0'] > 1.5) & (test['PAY_6'] <= 1.0) & (test['BILL_AMT1'] > 649.5), 1, 0)
test['Tree7'] = np.where((test['PAY_0'] > 1.5) & (test['PAY_6'] > 1.0) & (test['PAY_AMT3'] <= 14177.0), 1, 0)

In [4]:
## b) Splitting the train data-frame intro training (80%) and validation (20%) (taking into account the proportions of 0s and 1s)

## Defining the input and target variables
X = train.drop(columns = ['default payment next month'])
Y = train['default payment next month']

## Splitting the data
X_training, X_validation, Y_training, Y_validation = train_test_split(X, Y, test_size = 0.2, stratify = Y)

#### Random Forest:

In [5]:
## c) Using the top 7 variables from Exercise 15 part (e) in Exam 1 to  build a model on the training data-frame

## Redefining the input and target variables
X_training = X_training[['PAY_0', 'PAY_2', 'Most_Common', 'Tree2', 'Tree6', 'PAY_3', 'Tree7']]
X_validation = X_validation[['PAY_0', 'PAY_2', 'Most_Common', 'Tree2', 'Tree6', 'PAY_3', 'Tree7']]
test = test[['PAY_0', 'PAY_2', 'Most_Common', 'Tree2', 'Tree6', 'PAY_3', 'Tree7']]

## Building a Random Forest Model with default hyper-parameters
rf_md = RandomForestClassifier().fit(X_training, Y_training)

## Predicting on the validation set
rf_preds = rf_md.predict_proba(X_validation)[:, 1]

## Extracting estimated labels using the PRC function
rf_labels = prc.precision_recall_cutoff(Y_validation, rf_preds)

## Extracting optimal cutoff using the PRC function
rf_cutoff = prc.precision_recall_cutoff_cutoff(Y_validation, rf_preds)

## Reporting the optimal cutoff value
print('Optimal Cutoff of Random Forest Model:', round(rf_cutoff, 3))

## Reporting the F1-Score of the model
print('\nF1-Score of Random Forest Model:', round(f1_score(Y_validation, rf_labels) * 100, 2), '%')

Optimal Cutoff of Random Forest Model: 0.192

F1-Score of Random Forest Model: 51.99 %


In [6]:
## Tuning the Random Forest model on the validation data-frame

## Defining the parameter dictionary
rf_param_grid = {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [5, 10, 15], 
                  'min_samples_leaf': [5, 10, 15]}

## Running GridSearchCV with 3 folds
rf_grid_search = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(X_validation, Y_validation)

## Extracting the best hyper-parameters
print('Optimal hyper-parameters for Random Forest Model: \n', rf_grid_search.best_params_)
print('\nOptimal F1-Score:\n', round(rf_grid_search.best_score_ * 100, 2), '%')

Optimal hyper-parameters for Random Forest Model: 
 {'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 15, 'n_estimators': 300}

Optimal F1-Score:
 43.67 %


In [7]:
## Building a Random Forest model with the optimal hyper-parameters
rf_md = RandomForestClassifier(max_depth = 7, min_samples_leaf = 5, min_samples_split = 15, 
                               n_estimators = 300).fit(X_training, Y_training)

## Predicting on the validation set
rf_preds = rf_md.predict_proba(X_validation)[:, 1]

## Extracting estimated labels using the PRC function
rf_labels = prc.precision_recall_cutoff(Y_validation, rf_preds)

## Extracting optimal cutoff using the PRC function
rf_cutoff = prc.precision_recall_cutoff_cutoff(Y_validation, rf_preds)

## Reporting the optimal cutoff value
print('Optimal Cutoff of Random Forest Model:', round(rf_cutoff, 3))

## Reporting the F1-Score of the model
print('\nF1-Score of Random Forest Model:', round(f1_score(Y_validation, rf_labels) * 100, 2), '%')

Optimal Cutoff of Random Forest Model: 0.188

F1-Score of Random Forest Model: 52.34 %


In [8]:
## Finally, using the optimal model to predict the likelihood of default payment next month on the test

rf_test_preds = rf_md.predict_proba(test)[:, 1]

#### AdaBoost:

In [9]:
## d) Using the top 7 variables from Exercise 15 part (e) in Exam 1 to  build a model on the training data-frame

## Building an AdaBoost Model with default hyper-parameters
ada_md = AdaBoostClassifier().fit(X_training, Y_training)

## Predicting on the validation set
ada_preds = ada_md.predict_proba(X_validation)[:, 1]

## Extracting estimated labels using the PRC function
ada_labels = prc.precision_recall_cutoff(Y_validation, ada_preds)

## Extracting optimal cutoff using the PRC function
ada_cutoff = prc.precision_recall_cutoff_cutoff(Y_validation, ada_preds)

## Reporting the optimal cutoff value
print('Optimal Cutoff of AdaBoost Model:', round(ada_cutoff, 3))

## Reporting the F1-Score of the model
print('\nF1-Score of AdaBoost Model:', round(f1_score(Y_validation, ada_labels) * 100, 2), '%')

Optimal Cutoff of AdaBoost Model: 0.492

F1-Score of AdaBoost Model: 49.67 %


In [10]:
## Tuning the AdaBoost model on the validation data-frame

## Defining the parameter dictionary
ada_param_grid = {'n_estimators': [100, 300, 500], 'base_estimator__min_samples_split': [10, 15], 
                  'base_estimator__min_samples_leaf': [10, 15], 'base_estimator__max_depth': [3, 5, 7], 
                  'learning_rate': [0.001, 0.01, 0.1]}

## Running GridSearchCV with 3 folds
ada_grid_search = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), ada_param_grid, 
                               cv = 3, scoring = 'f1', n_jobs = -1).fit(X_validation, Y_validation)

## Extracting the best hyper-parameters
print('Optimal hyper-parameters for AdaBoost Model: \n', ada_grid_search.best_params_)
print('\nOptimal F1-Score:\n', round(ada_grid_search.best_score_ * 100, 2), '%')

Optimal hyper-parameters for AdaBoost Model: 
 {'base_estimator__max_depth': 3, 'base_estimator__min_samples_leaf': 15, 'base_estimator__min_samples_split': 10, 'learning_rate': 0.1, 'n_estimators': 100}

Optimal F1-Score:
 43.69 %


In [11]:
## Building a AdaBoost model with the optimal hyper-parameters
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(min_samples_split = 10, min_samples_leaf = 15, max_depth = 3), 
                            n_estimators = 100, learning_rate = 0.1).fit(X_training, Y_training)

## Predicting on the validation set
ada_preds = ada_md.predict_proba(X_validation)[:, 1]

## Extracting estimated labels using the PRC function
ada_labels = prc.precision_recall_cutoff(Y_validation, ada_preds)

## Extracting optimal cutoff using the PRC function
ada_cutoff = prc.precision_recall_cutoff_cutoff(Y_validation, ada_preds)

## Reporting the optimal cutoff value
print('Optimal Cutoff of AdaBoost Model:', round(ada_cutoff, 3))

## Reporting the F1-Score of the model
print('\nF1-Score of AdaBoost Model:', round(f1_score(Y_validation, ada_labels) * 100, 2), '%')

Optimal Cutoff of AdaBoost Model: 0.465

F1-Score of AdaBoost Model: 52.38 %


In [12]:
## Finally, using the optimal model to predict the likelihood of default payment next month on the test

ada_test_preds = ada_md.predict_proba(test)[:, 1]

#### XGBoost:

In [13]:
## e) Using the top 7 variables from Exercise 15 part (e) in Exam 1 to  build a model on the training data-frame

## Building XGBoost Model with default hyper-parameters
xgb_md = XGBClassifier(eval_metric = 'error', use_label_encoder = False).fit(X_training, Y_training)

## Predicting on the validation set
xgb_preds = xgb_md.predict_proba(X_validation)[:, 1]

## Extracting estimated labels using the PRC function
xgb_labels = prc.precision_recall_cutoff(Y_validation, xgb_preds)

## Extracting optimal cutoff using the PRC function
xgb_cutoff = prc.precision_recall_cutoff_cutoff(Y_validation, xgb_preds)

## Reporting the optimal cutoff value
print('Optimal Cutoff of XGBoost Model:', round(xgb_cutoff, 3))

## Reporting the F1-Score of the model
print('\nF1-Score of XGBoost Model:', round(f1_score(Y_validation, xgb_labels) * 100, 2), '%')

Optimal Cutoff of XGBoost Model: 0.199

F1-Score of XGBoost Model: 51.95 %


In [14]:
## Tuning the XGBoost model on the validation data-frame

## Defining the parameter dictionary
xgb_param_grid = {'n_estimators': [300, 500], 'max_depth': [5, 7], 'min_child_weight': [5, 7], 
                  'learning_rate' : [0.01, 0.001], 'gamma': [0.1, 0.01], 'subsample': [0.8, 1], 
                  'colsample_bytree': [0.8, 1], 'early_stopping_rounds': [100]}

## Running GridSearchCV with 3 folds
xgb_grid_search = GridSearchCV(XGBClassifier(eval_metric = 'error', use_label_encoder = False), xgb_param_grid, cv = 3, scoring = 'f1', 
                               n_jobs = -1).fit(X_validation, Y_validation)

## Extracting the best hyper-parameters
print('Optimal hyper-parameters for XGBoost Model: \n', xgb_grid_search.best_params_)
print('\nOptimal F1-Score:\n', round(xgb_grid_search.best_score_ * 100, 2), '%')

KeyboardInterrupt: 

In [ ]:
## Building a XGBoost model with the optimal hyper-parameters
xgb_md = XGBClassifier(n_estimators = , max_depth = , min_child_weight = , learning_rate = ,
                      gamma = , subsample = , colsample_bytree = , early_stopping_rounds = ).fit(X_training, Y_training)

## Predicting on the validation set
xgb_preds = xgb_md.predict_proba(X_validation)[:, 1]

## Extracting estimated labels using the PRC function
xgb_labels = prc.precision_recall_cutoff(Y_validation, xgb_preds)

## Extracting optimal cutoff using the PRC function
xgb_cutoff = prc.precision_recall_cutoff_cutoff(Y_validation, xgb_preds)

## Reporting the optimal cutoff value
print('Optimal Cutoff of XGBoost Model:', round(xgb_cutoff, 3))

## Reporting the F1-Score of the model
print('\nF1-Score of XGBoost Model:', round(f1_score(Y_validation, xgb_labels) * 100, 2), '%')

In [ ]:
## Finally, using the optimal model to predict the likelihood of default payment next month on the test

xgb_test_preds = xgb_md.predict_proba(test)[:, 1]

#### Meta-Learner

In [ ]:
## f) Using the predictions on the validation data-frame as inputs from parts (c)-(d)-(e) and the actual default payment 
## next month values from the validation data-frame to build a meta-learner to predict default payment next month

## Building the meta-learner data frame
meta_data = pd.Dataframe({'rf': rf_preds, 'ada': ada_preds, 'xgb': xgb_preds, 'Y_validation': Y_validation})

## Building the meta-learner test data frame
meta_data_test = pd.Dataframe({'rf': rf_test_preds, 'ada': ada_test_preds, 'xgb': xgb_test_preds})

## Defining the input and target variables
X_meta = meta_data[['rf', 'ada', 'xgb']]
Y_meta = meta_data['Y_validation']

## Defining the parameter dictionary
meta_param_grid = {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [5, 10, 15], 
                  'min_samples_leaf': [5, 10, 15]}

## Running GridSearchCV with 3 folds
meta_grid_search = GridSearchCV(RandomForestClassifier(), meta_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(X_meta, Y_meta)

## Extracting the best hyper-parameters
print('Optimal hyper-parameters for Meta-Learner Random Forest Model: \n', meta_grid_search.best_params_)
print('\nOptimal F1-Score:\n', round(meta_grid_search.best_score_ * 100, 2), '%')

In [ ]:
## Splitting the data
X_meta_training, X_meta_validation, Y_meta_training, Y_meta_validation = train_test_split(X_meta, Y_meta, 
                                                                                          test_size = 0.2, stratify = Y_meta)

## Building a Meta-Learner model with the optimal hyper-parameters
meta_md = RandomForestClassifier(n_estimators = , max_depth = , min_samples_split = , 
                                 min_samples_leaf = ).fit(X_meta_training, Y_meta_training)

## Predicting on the validation set
meta_preds = meta_md.predict_proba(X_meta_validation)[:, 1]

## Extracting estimated labels using the PRC function
meta_labels = prc.precision_recall_cutoff(Y_meta_validation, meta_preds)

## Extracting optimal cutoff using the PRC function
meta_cutoff = prc.precision_recall_cutoff_cutoff(Y_meta_validation, meta_preds)

## Reporting the optimal cutoff value
print('Optimal Cutoff of Meta-Learner Model:', round(meta_cutoff, 3))

## Reporting the F1-Score of the model
print('\nF1-Score of Meta-Learner Model:', round(f1_score(Y_meta_validation, meta_labels) * 100, 2), '%')

In [ ]:
## Finally, using the optimal meta-learner to predict the likelihood of default payment next month on the test
meta_test_preds = meta_md.predict_proba(meta_data_test)[:, 1]

## Creating a data-frame to store results
meta_likelihoods = pd.DataFrame({'Likelihoods': meta_test_preds})

## Exporting predicted likelihoods as a csv file
meta_likelihoods.to_csv('meta_likelihoods.csv', index = False)

## Associated cutoff value = 0.